In [90]:
# Работа с изображениями и текстом

from PIL import Image, ImageDraw, ImageFont

out = Image.new("RGB", (150, 100), (255, 255, 255))
d = ImageDraw.Draw(out)
d.line([0,0, 80,80], fill=128)
d.multiline_text((10,10), "Hello\nWorld", font=d.getfont(), fill=(0, 200, 0))
out.save('newImage.png', 'png')

In [10]:
# Загрузка окра, запускать при смене документа.  

import urllib3
import json

import config

http = urllib3.PoolManager()
r = http.request('GET', config.ocr_href)
ocrData = r.data.decode('utf-8')
f = open('./tmp/ocrData.csv', 'w')
f.write(ocrData)
f.close()

In [11]:
# Обновление файла с картинкой документа (очистка всего что нарисовалось локально).  

import urllib3

import config

http = urllib3.PoolManager()
r = http.request('GET', config.image_href)
f = open(config.image_name, 'wb')
f.write(r.data)
f.close()


In [12]:
import numpy as np
import pandas as pn
import math
from PIL import Image, ImageDraw, ImageFont

import config

ocr = pn.read_csv('./tmp/ocrData.csv')

# Точка начала базиса.  

one_point = ocr.query("text == 'Указания'")

# Вспомогательные вектора (в принципе достаточно одного, для масштабирования использую А, второй для быстрой замены при тестах).  

a_point = frameOcr.query("text == 'Прием'")
b_point = frameOcr.query("text == 'Сдача'")


with Image.open("test.png") as doc_image:

    draw = ImageDraw.Draw(doc_image)
    basis = config.basis
    ofset_x = one_point.iloc[0]['left']
    ofset_y = one_point.iloc[0]['top']

    # Определение коэффициента масштабирования.  
    scale_x = (b_point.left - ofsetX) / basis['B']['x']
    scale_y = (b_point.top - ofsetY) / basis['B']['y']

    # Координаты точки начала   
    basis_position = [ofset_x, ofset_y]
    
    for point in basis:
        vector = basis[point]
        vector = [
            vector['x'] * scale_x + ofset_x,
            vector['y'] * scale_y + ofset_y
        ]
        draw.line(basis_position + vector, fill=128, width=10)


    doc_image.save("test.png", "PNG")